In [4]:
import pandas as pd
import numpy as np
from openai import OpenAI
import os
from dotenv import load_dotenv

In [7]:
# --- 1. CONFIGURACIÓN ---
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
df_merged = pd.read_csv('./archivos/dataset_merged.csv') # Carga tu DF aquí

# El cliente de OpenAI busca automáticamente la clave en la variable de entorno 
# "OPENAI_API_KEY" si no le pasas el 'api_key' como argumento.
try:
    client = OpenAI() 
except Exception as e:
    print("❌ Error al inicializar el cliente de OpenAI. Asegúrate de que 'OPENAI_API_KEY' esté en el archivo .env.")
    print(e)
    # Finaliza el script o define un cliente dummy si falla
    # client = None


# --- 2. PREPARACIÓN DE PELÍCULAS ÚNICAS (ETL) ---
# Creamos un catálogo único para no gastar dinero vectorizando la misma peli 2 veces
unique_movies = df_merged.drop_duplicates(subset=['movieId']).copy()

# Preparamos el texto enriquecido para el embedding
# Combinamos Título + Géneros + Descripción para que la IA entienda el contexto total
unique_movies['text_features'] = (
    "Title: " + unique_movies['title'].astype(str) + "; " +
    #"Genres: " + unique_movies['genres'].astype(str) + "; " +
    "Overview: " + unique_movies['overview'].astype(str)
)

print(f"Total de películas únicas a vectorizar: {len(unique_movies)}")


# --- 3. GENERACIÓN DE EMBEDDINGS (Batch) ---
# Función para obtener embeddings
def get_embedding(text, model="text-embedding-3-small"):
    # Limpieza simple de saltos de línea
    text = str(text).replace("\n", " ")
    # Retorna el vector (lista de floats)
    return client.embeddings.create(input=[text], model=model).data[0].embedding

# IMPORTANTE: Si tienes muchas películas (>2000), haz esto por lotes o usa una muestra primero.
# Para este ejemplo, tomaremos una muestra de 100 pelis únicas para que corra rápido.
# En producción, quita el .head(100)
#unique_movies_subset = unique_movies.head(100).copy() 

print("Generando embeddings con OpenAI...")
unique_movies['embedding'] = unique_movies['text_features'].apply(lambda x: get_embedding(x))
print("Embeddings listos.")



Total de películas únicas a vectorizar: 2830
Generando embeddings con OpenAI...
Embeddings listos.


In [8]:
embedding_movies_csv=unique_movies.to_csv('./archivos/embedding_movies.csv', index=False)

print(f"✅ DataFrame guardado exitosamente")
print("Este archivo ahora contiene las columnas de la película más el vector 'embedding'.")

✅ DataFrame guardado exitosamente
Este archivo ahora contiene las columnas de la película más el vector 'embedding'.
